### 决策树

In [1]:
# 西瓜数据集
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
plt.style.use('ggplot')
%matplotlib inline
# 对中文的支持
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [3]:
df = pd.read_csv('./data_watermelon/watermelon.data')

In [4]:
df

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,0.243,0.267,否


In [5]:
# 求信息熵
def ent(data,a):
    info_ent = {}  # 信息熵
    root = '好瓜'
    # 求根节点信息熵
    if a==root:
        pi = data[root].value_counts()/len(data)
        return np.sum(-pi*np.log2(pi))
    
    # 子节点信息熵
    a_val = data[a].value_counts()      # 属性 a = {D1,D2,..,Dv}
    a_index = a_val.index               # example-> ['乌黑', '青绿', '浅白']
    
    for d in a_index:
        # d = '乌黑'...
        temp_data = data[data[a]==d]
        pi = temp_data[root].value_counts()/len(temp_data)
        info_ent[d] =  np.sum(-pi*np.log2(pi))
    return info_ent

In [751]:
# 求信息增益
def gain(data,a):
    info_gain = ent(data,'好瓜') # 信息增益
    info_ent = ent(data,a)
    Pd = data[a].value_counts()/len(data[a])         # Dv/D
    for k,v in info_ent.items():
        info_gain -= Pd[k]*v
    return info_gain

In [796]:
class tnode:
    def __init__(self,n_name='',n_branch=[]):
        self.name = n_name
        self.type = ''
        self.branch = n_branch
        self.next = []
    
#
def treeG(dt,A,node):
    # 默认最后一行为标签
    Y = dt.iloc[:,-1].value_counts()#.index
    # 属性集 A = {a1,a2,..,ad}
    index_A = list(A)
    
    # 生成node节点

    # 如果样本属于同一类别 C，则返回
    if len(Y)==1:
        # 将 节点标记为 类别 C
#         node.type = Y.index
        return '无需划分'
    # 如果 A 属性集为空或只有一个属性
    if len(index_A)==0: #or len(index_A)==1:
#         node.name = index_A
        m = dt.iloc[:,-1].value_counts()
        node.type = m[m==max(m)].index
        return '无法划分'
    
    # 从 A 中选择最优划分属性 即 求最大信息熵对应的属性 a*
    # gain_max = max([gain(dt,ad) for ad in index_A])
    gain_list = []
    for ad in index_A:
        gain_list.append(gain(dt,ad))
#     print(gain_list)
    max_a = index_A[np.argmax(gain_list)]
    # 根据属性 a* 分类数据集然后遍历 eg: a_v in ['清晰','稍糊','模糊']
#     node.next.append(tnode(max_a))
#     note = node.next
    
    print(max_a)
    max_a_v = dt[max_a].value_counts().index
#     print(max_a_v)
    index_A.pop(np.argmax(gain_list))
    for a_v in max_a_v:
#         node.branch.append(a_v)
        Dv = dt[dt[max_a]==a_v][index_A+[Y.name]]
#         print(Dv)
        print('-->'+ a_v)
        if a_v =='浅白':
            print(a_v)
        if len(Dv)==0:
            node.next[-1].name = ''
            m = dt.iloc[:,-1].value_counts()
#             node.type = m[m==max(m)].index
            print('Dv is null')
            return '---'
        else:
            d = treeG(Dv,index_A,node)
        print(d)
        

In [797]:
dt = df.iloc[:,[1,2,3,4,5,6,-1]]
A = list(df.columns[[1,2,3,4,5,6]])
Y = '好瓜'

In [800]:
dt

,色泽,根蒂,敲声,纹理,脐部,触感,好瓜
0,青绿,蜷缩,浊响,清晰,凹陷,硬滑,是
1,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,是
2,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,是
3,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,是
4,浅白,蜷缩,浊响,清晰,凹陷,硬滑,是
5,青绿,稍蜷,浊响,清晰,稍凹,软粘,是
6,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,是
7,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,是
8,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,否
9,青绿,硬挺,清脆,清晰,平坦,软粘,否


In [799]:
node = tnode()
# ent(dt,'色泽')
x = treeG(dt,A,node)

纹理
-->清晰
根蒂
-->蜷缩
无需划分
-->稍蜷
色泽
-->乌黑
触感
-->软粘
无需划分
-->硬滑
无需划分
None
-->青绿
无需划分
None
-->硬挺
无需划分
None
-->稍糊
触感
-->硬滑
无需划分
-->软粘
无需划分
None
-->模糊
无需划分
